### Zadanie 1

Uruchomić:

* naive_multiplication(A,B),
* better_multiplication(A,B)
* mnożenie BLAS w Julii (A*B)

dla coraz większych macierzy kwadratowych i zmierzyć czasy. Narysować wykres zależyności czasu od rozmiaru macierzy wraz z słupkami błędów, tak jak na poprzednim laboratorium. Wszystkie trzy metody powinny być na jednym wykresie. (1pkt)


In [1]:
function naive_multiplication(A,B)
    C = zeros(Float64, size(A,1), size(B,2))
    for i=1:size(A,1)
        for j=1:size(B,2)
            for k=1:size(A,2)
                C[i,j] = C[i,j] + A[i,k]*B[k,j]
            end
        end
    end
end


function better_multiplication(A, B)
    C = zeros(Float64, size(A,1), size(B,2))
    for j=1:size(B,2)
        for k=1:size(A,2)
            for i=1:size(A,1)
                C[i,j] = C[i,j] + A[i,k]*B[k,j]
            end
        end
    end
end


function naive_time(A, B)
    @elapsed naive_multiplication(A, B)
end

function better_time(A, B)
    @elapsed better_multiplication(A, B)
end

function blas_time(A, B)
    @elapsed A*B
end

blas_time (generic function with 1 method)

In [ ]:
using DataFrames

dataframe= DataFrame(Size = 1, Operation="NaiveTime", Time = 0.0)
delete!(dataframe, 1)

functions = [naive_time, better_time, blas_time]
texts = ["naive_multiplication", "better_multiplication", "BLAS_multiplication"]

for n = 10:100:1200
    for i in 1:3
        func = functions[i]
        text = texts[i]
        A = rand(n, n)
        B = rand(n, n)
        for a = 1:10
            push!(dataframe, [n, text, func(A, B)])
        end
    end
end

dataframe

In [ ]:
using Statistics

grouped = groupby(dataframe, [:Operation, :Size])
aggregated = combine(grouped, "Time" => mean, "Time" => std)

In [ ]:
using Plots

scatter(aggregated.Size, aggregated.Time_mean, yerror=aggregated.Time_std, group=aggregated.Operation, 
        legend=:topleft, markersize=[3 3 3], ylabel="Time [s]", xlabel="Matrix size", title="Julia matrix operations")

### Zadanie 2
Napisać w języku C:

* naiwną metodę mnożenia macierzy kwadratowych (wersja 1)

* ulepszoną wersję za pomocą zamiany pętli metodę mnożenia macierzy(wersja 2), pamiętając, że w C macierz przechowywana jest wierszami (row major order tzn A11,A12, ..., A1m, A21, A22,...,A2m, ..Anm), inaczej niż w Julii !

* skorzystać z możliwości BLAS poziom 3 dostępnego w GSL (przykład uzycia https://www.gnu.org/software/gsl/doc/html/blas.html#examples )

Należy porównywać działanie tych trzech algorytmow bez włączonej opcji optymalizacji kompilatora. Przedstawić wyniki na jednym wykresie tak jak w p.1. (osobno niż p.1). (1 pkt)

(Dla chętnych) sprawdzić, co się dzieje, jak włączymy optymalizację kompilatora i dodać do wykresu.


```c
#include <stdio.h>
#include <stdlib.h>
#include <gsl/gsl_blas.h>
#include <float.h>
#include <sys/times.h>
#include <unistd.h>

clock_t start_time, end_time;
struct tms start_tms, end_tms;

void start_the_time() {
    start_time = times(&start_tms);
}

void end_the_time() {
    end_time = times(&end_tms);
}

double get_time() {
    return (double) (end_time - start_time) / sysconf(_SC_CLK_TCK);
}

void naive_multiply(double** A, double** B, double** C, int n){
    for (int i = 0; i < n; i++)
        for (int j = 0; j < n; j++)
            for (int k = 0; k < n; k++)
                C[i][j] += A[i][k] * B[k][j];
}

void better_multiply(double** A, double** B, double** C, int n){
    for (int i = 0; i < n; i++)
        for (int k = 0; k < n; k++)
            for (int j = 0; j < n; j++)
                C[i][j] += A[i][k] * B[k][j];
}

void free_matrix(double** matrix, int n) {
    for (int i = 0; i < n; i++) {
        free(matrix[i]);
    }
    free(matrix);
}

double** random_matrix(int n) {
    double** matrix = malloc(n * sizeof(double*));
    for (int i = 0; i < n; i++) {
        matrix[i] = malloc(n * sizeof(double));
        for (int j = 0; j < n; j++) {
            matrix[i][j]  = (double) rand() / (double) RAND_MAX;
        }
    }
    return matrix;
}

gsl_matrix* random_gsl(int n) {
    gsl_matrix* matrix = gsl_matrix_alloc(n, n);
    for (int i = 0; i < n; i++) {
        for (int j = 0; j < n; j++) {
            gsl_matrix_set(matrix, i, j, (double) rand() / RAND_MAX);
        }
    }
    return matrix;
}

int main() {
    FILE* file = fopen("data.csv", "w");

    fprintf(file, "Size,Operation,Time");
    double time;

    for (int i = 10; i < 500; i += 90) {
        for (int j = 0; j < 10; j++) {
            double** A = random_matrix(i);
            double** B = random_matrix(i);
            double** C = random_matrix(i);

            start_the_time();
            naive_multiply(A, B, C, i);
            end_the_time();
            time = get_time();
            fprintf(file, "\n%d,%s,%.*e", i, "naive_multiplication", DECIMAL_DIG, time);

            start_the_time();
            better_multiply(A, B, C, i);
            end_the_time();
            time = get_time();
            fprintf(file, "\n%d,%s,%.*e", i, "better_multiplication", DECIMAL_DIG, time);

            free_matrix(A, i);
            free_matrix(B, i);
            free_matrix(C, i);

            gsl_matrix* a = random_gsl(i);
            gsl_matrix* b = random_gsl(i);
            gsl_matrix* c = random_gsl(i);
            start_the_time();
            gsl_blas_dgemm(CblasNoTrans, CblasNoTrans, 1.0, a, b, 0.0, c);
            end_the_time();
            time = get_time();
            fprintf(file, "\n%d,%s,%.*e", i, "GSL_multiplication", DECIMAL_DIG, time);

            gsl_matrix_free(a);
            gsl_matrix_free(b);
            gsl_matrix_free(c);
        }
    }
    fclose(file);
    return 0;
}
```

In [ ]:
using CSV

c_dataframe = CSV.read("c_data.csv", delim=",", DataFrame)

In [ ]:
c_grouped = groupby(c_dataframe, [:Operation, :Size])
c_aggregated = combine(c_grouped, "Time" => mean, "Time" => std)

In [ ]:
scatter(c_aggregated.Size, c_aggregated.Time_mean, yerror=c_aggregated.Time_std, group=c_aggregated.Operation, 
        legend=:topleft, markersize=[4 4 4], ylabel="Time [s]", xlabel="Matrix size", title="C matrix operations")

### Zadanie 3
Użyć funkcji polyfit z pakietu Polynomials do znalezienia odpowiednich wielomianów, które najlepiej pasują do zależności czasowych każdego z algorytmów. Stopień wielomianu powinien zgadzać się z teoretyczną złożonoscią. Dodać wykresy uzyskanych wielomianów do wcześniejszych wykresów. (1 pkt)

In [ ]:
using Polynomials

values = groupby(c_aggregated, :Operation)
args = values[1].Size

for i = 1:1:3
    tmp_values = values[i].Time_mean
    fit_function = fit(args, tmp_values, 3)
    plot!(fit_function, extrema(args)..., label=texts[i])
end
plot!()

In [ ]:
values = groupby(aggregated, :Operation)
args = values[1].Size

scatter(aggregated.Size, aggregated.Time_mean, yerror=aggregated.Time_std, group=aggregated.Operation, 
        legend=:topleft, markersize=[3 3 3], ylabel="Time [s]", xlabel="Matrix size", title="Julia matrix operations")

for i = 1:1:3
    tmp_values = values[i].Time_mean
    fit_function = fit(args, tmp_values, 3)
    plot!(fit_function, extrema(args)..., label=texts[i])
end
plot!()

### Zadanie 4
Pokazać zniwelowanie efektu Rungego (przy interpolacji) poprzez użycie wsparcia dla wielomianów Czebyszewa w pakiecie Polynomials. Narysować wybraną funkcję z zaznaczonymi węzłami i wielomianem interpolacyjnym dla węzłów równoodległych oraz Czebyszewa (2 wykresy).(1 pkt)

In [ ]:
function runge(x)
    return 1 / (1 + 25 * x * x)
end

xs = -1:0.005:1.0
ys = [runge(x) for x in xs]

scatter(xs, ys, label="Runge function", title="Even distribution of nodes")

interX = -1:0.2:1
interY = [runge(x) for x in interX]
f = fit(interX, interY)
plot!(f,  extrema(interX)..., label="polynomial interpolation with 11 nodes", width=4)

scatter!(interX, interY, label="Nodes")

In [ ]:
using ChebyshevApprox

xs = -1:0.005:1.0
ys = [runge(x) for x in xs]

# scatter(xs, ys, label="Runge function")

# nodes = chebyshev_nodes(11)
# values = [runge(x) for x in nodes]
# f = fit(nodes, values)
# plot!(f,  extrema(nodes)..., label="Chebyshev interpolation with 11 nodes", width=4)

# scatter!(nodes, values, label="Nodes")

chebyshev_base = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]
chebyshev_poly = ChebyshevT(chebyshev_base)
args = Polynomials.roots(chebyshev_poly)
values = [runge(x) for x in args]

scatter(xs, ys, label="Runge function", title="Chebyshev nodes")
poly_fit = fit(args, values)
plot!(poly_fit, extrema(args)..., label="Chebyshev interpolation", width=4)

scatter!(args, values, label="nodes")

### Zadanie 5
Przybliżenie Pade jest często lepsze niż rozwinięcie w szereg Taylora przy aproksymowaniu funkcji, które posiadają osobliwości. Korzystając ze wsparcia dla aproksymacji Pade w pakiecie Polynomials pokazać dowolny przykład (wraz z wykresem), gdzie takie przybliżenie faktycznie jest lepsze. Można odtworzyć wykres z wykładu albo zainspirować się przykładowym artykułem https://www.hindawi.com/journals/ijcm/2014/587430/ (1 pkt)

In [ ]:
using TaylorSeries

function example_Pade(x)
    return log(1 + x) / x
end

taylor = Taylor1(Float64, 5)
taylor_func = example_Pade(taylor)
taylor_poly = Polynomial(taylor_func.coeffs)

xs = 1:0.05:5.0
ys = [example_Pade(x) for x in xs]

scatter(xs, ys, label="Example function", ylims=[0, 4])
plot!(taylor_poly, extrema(xs)..., label="Taylor approximation", width=3, color=:black)

pade_object = Polynomials.PolyCompat.PadeApproximation.Pade(taylor_poly, 2, 2)
pade_poly(x) = pade_object.p(x) / pade_object.q(x)

plot!(pade_poly, extrema(xs)..., label="Pade approximation", width=3, color=:red)